In [23]:
from dataclasses import dataclass

from pydantic_ai import Agent, RunContext

from pydanticai_examples.config import get_qdrant_config, get_openai_config
from pydanticai_examples.vector_store import VectorStore
from dotenv import load_dotenv
from pydanticai_examples.utils import get_root


In [24]:
load_dotenv(get_root() / ".env")

True

In [25]:
@dataclass
class ChosenRecipe:
    """Class to represent a chosen recipe"""
    name: str
    description: str
    ingredients: list[str]
    instructions: str

    def __str__(self):
        return f"{self.name}: {self.description}\nIngredients: {', '.join(self.ingredients)}\nInstructions: {self.instructions}"

In [26]:
cooking_agent = Agent[VectorStore, ChosenRecipe](
    model='openai:gpt-4-turbo',
    deps_type=VectorStore,
    result_type=ChosenRecipe,
    system_prompt="""You are a professional dietitian. For a given list of ingredients that a user has in their stash,
    you should suggest a recipe that uses those ingredients.
    You should only use recipes that are contained in the vector database.
    """
)

In [ ]:
@cooking_agent.system_prompt
async def include_user_name(ctx: RunContext[str]) -> str:
    """Include the user's name in the system prompt"""
    return f"The user name is {ctx.deps}"

In [27]:

@cooking_agent.tool
async def vector_search(ctx: RunContext[VectorStore], ingredients: list[str]) -> list[dict]:
    """Search the vector store for cooking recipes including the ingredients provided.

    Args:
        ingredients (list[str]): The list of ingredients the resulting
            recipes should include.

    Returns:
        list[str]: List of recipes found for a given query.
    """
    # Embed the query
    ingredients_str = ", ".join(ingredients)
    return await ctx.deps.search(ingredients_str)

In [28]:
# Instantiate the vector store
qdrant_config = get_qdrant_config()
openai_config = get_openai_config()
vector_store = VectorStore(
    url=qdrant_config.qdrant_url,
    port=qdrant_config.qdrant_port,
    collection_name=qdrant_config.qdrant_index_name,
    openai_api_key=openai_config.openai_api_key,
)

In [29]:
result = await cooking_agent.run("I have only milk and wheat", deps=vector_store)

In [21]:
result.data

ChosenRecipe(name='Milk and Egg Fried Bread', description='A simple and delicious recipe that uses basic ingredients such as milk, butter, and bread. This dish is perfect for a quick breakfast or a comforting snack.', ingredients=['milk', 'butter', 'bread', 'sugar', 'cinnamon (optional)'], instructions='1. Beat the eggs and milk in a bowl.\n2. Add the sugar and cinnamon.\n3. Take one slice of bread and place it in the egg mixture, soak it on both sides for 30 seconds.\n4. Meanwhile, melt the butter in the pan, making sure it doesn’t begin to brown.\n5. Add the bread and fry for around 2 minutes on each side.\n6. Optionally, leave out the sugar and have it with honey, jam, or syrup.')

In [9]:
result.all_messages()

[ModelRequest(parts=[SystemPromptPart(content='You are a professional dietitian. For a given list of ingredients that a user has in their stash,\nyou should suggest a recipe that uses those ingredients.\nYou should only use recipes that are contained in the vector database.\n    ', timestamp=datetime.datetime(2025, 3, 30, 13, 36, 32, 841600, tzinfo=datetime.timezone.utc), dynamic_ref=None, part_kind='system-prompt'), UserPromptPart(content='I have only milk and wheat', timestamp=datetime.datetime(2025, 3, 30, 13, 36, 32, 841608, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request'),
 ModelResponse(parts=[ToolCallPart(tool_name='search', args='{"ingredients":["milk","wheat"]}', tool_call_id='call_Ce58s7l7wKEE1Cocarxg9Wbn', part_kind='tool-call')], model_name='gpt-4-turbo-2024-04-09', timestamp=datetime.datetime(2025, 3, 30, 13, 36, 33, tzinfo=datetime.timezone.utc), kind='response'),
 ModelRequest(parts=[ToolReturnPart(tool_name='search', content=[ScoredPoint(id='aeb

In [22]:
result.usage()

Usage(requests=2, request_tokens=3066, response_tokens=213, total_tokens=3279, details={'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0, 'cached_tokens': 0})